In [1]:
import os
import sys
import time
import argparse
import numpy as np
from tqdm import tqdm
sys.path.append('..')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

from scripts.dataset import ImgDataset, readLabel, readImg
from scripts.models import DG_model

In [ ]:
workspace = 'data'
task = 'siw'
if task == 'oulu':
    workspace_dir = os.path.join(workspace, 'oulu_npu_cropped/test')
    test_x = np.load(os.path.join(workspace,'oulu_npu_cropped/test_x.npy'))
if task == 'siw':
    workspace_dir = os.path.join(workspace, 'siw_test')
    test_x = np.load(os.path.join(workspace, 'test_siw_x.npy'))

In [3]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]),
])
batch_size = 60
test_set = ImgDataset(test_x, None, None,test_transform, 'group')
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
binary = False
model = DG_model(model = 'resnet18', num_cls = (1 if binary else 3)).cuda()
model_path = '../logdir/DG_model_01-12-14:05:26/models/model-19-0.964747-0.993563.pth'
model.load_state_dict(torch.load(model_path))
_ = model.eval()

In [8]:
test_preds = []
with torch.no_grad():
    for vids in tqdm(test_loader):
        for vid in vids:
            class_logits, embedded = model(vid.cuda())
            if binary:
                class_logits = torch.mean(class_logits.squeeze()).cpu().numpy()
            else:
                class_logits = np.argmax(class_logits.cpu().detach().numpy(), axis=1)
                counts = np.bincount(class_logits)
                class_logits = np.argmax(counts)
            test_preds.append(class_logits)

100%|██████████| 35/35 [00:27<00:00,  1.27it/s]


In [12]:
import pandas as pd
result = np.array(test_preds)
image_dir = sorted(os.listdir(workspace_dir))
df = pd.DataFrame({'video_id':image_dir, 'label': result})
df.to_csv('{}.csv'.format(task),index=False)